# Formatted zone

In [1]:
import os, sys
import pandas as pd
import duckdb 

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ipynb.fs.full.utils import df_to_DBtable, get_tables, DBtable_to_df

## Repositories

### Household repository

In [2]:
household_columns = ['section','delete','population','single_women_aged_16_to_64','single_men_aged_16_to_64','single_women_aged_65_or_over','single_men_aged_65_or_over',
    'adult_women_with_one_or_more_minors','adult_men_with_one_or_more_minors','two_adults_from_16_to_64_and_without_minors',
    'two_adults_one_at_least_65_and_without_minors','two_adults_and_one_minor','two_adults_and_two_minors','two_adults_and_three_or_more_minors',
    'two_adults_over_35_and_one_adult_from_16_to_34','two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor',
    'two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors','three_adults_and_0_or_more_minors','two_adults_over_35_and_two_adults_from_16_to_34',
    'two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor','two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors',
    'four_adults_and_0_or_more_minors','five_adults_and_0_or_more_minors','fifteen_or_more_inhabitants','only_minors']

def read_household(file):
    """
    Prepares the Household excel file, 
    Returns the data in DataFrame format
    """
    path = f'../1.Landing zone/persistent/{file}'
    df = pd.read_excel(path,sheet_name='Composicion del hogar', header=[5], names=household_columns)

    # Formatting the excel format to dataframe
    df['section'].fillna(df['delete'],inplace=True)
    df.drop(labels='delete', axis=1, inplace=True)
    df.dropna(inplace = True)

    # Removes the total rows 
    newDF = df[pd.to_numeric(df['section'], errors='coerce').notnull()]
    assert df.shape[0] - newDF.shape[0] == 22

    return newDF

### Nationalities repsitory

In [3]:

def read_nationalities(file):
    """
    Reads the Nationalities Excel
    Returns the data in DataFrame format
    """
    path = f'../1.Landing zone/persistent/{file}'
    df = pd.read_excel(path, sheet_name='Total', header=[7])

    # Formatting excel format to dataframe
    df.rename(columns = {'Unnamed: 0':'Madrid_section','Unnamed: 2':'Habitantes','Unnamed: 3':'Españoles','Unnamed: 4':'Extranjeros'}, inplace = True)
    df.drop('Unnamed: 1', axis=1, inplace=True)
    df.drop(df.filter(regex='Unname'),axis=1, inplace=True)
    df.dropna(inplace = True)

    # Removes the total columns
    df.drop(df.filter(like='Total'),axis=1, inplace=True)

    # Formatting the column name
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')

    # Removes the total rows 
    newDF = df[df['Madrid_section'].apply(lambda x: len(x.strip()) == 9)]

    return newDF

## Main

In [4]:
def formatted_zone():
    """
    Stores the excel tables in a relational data base.
    """
    for file in os.listdir('../1.Landing zone/persistent/'):  
        table = file.split('_')[1].split('.')[0]
        repo = ''.join(filter(str.isalpha,table))
        df = read_household(file) if repo == 'household' else read_nationalities(file)
        df_to_DBtable(f'../{repo}.duckdb',df,table)


In [5]:
formatted_zone()

### Test

In [6]:
# check if all tables in household.duckdb are created
get_tables('../household.duckdb')

0    household2018
1    household2020
2    household2019
Name: table_name, dtype: object

In [7]:
# check if all tables in nationalities.db are created
get_tables('../nationalities.duckdb')

0    nationalities2018
1    nationalities2020
2    nationalities2019
Name: table_name, dtype: object